## Introduction

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

In [8]:
# List of packages required

import pandas as pd
!pip install wikipedia
import wikipedia as wp
from bs4 import BeautifulSoup
!pip install lxml
import lxml
!pip install geocoder
import geocoder # import geocoder


import requests
import io

print("Packages are imported")

Packages are imported


## Preprocessing

### Use wikipedia package to read the Table

In [25]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

# determine the index of your table
df = pd.read_html(html, header = 0)[0]

# Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']


### Criteria 1

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: **Harbourfront and Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [26]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].agg(lambda x : ','.join(set(x))).to_frame().reset_index()

### Criteria 2

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [27]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

In [28]:
# print the Shape of the table

print('The number of rows in Postal codes of Canada Table is ' , df.shape[0])

The number of rows in Postal codes of Canada Table is  103


In [29]:
df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [30]:
# Read geographical coordinates of each postal code from a csv file

url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
coordinates=pd.read_csv(io.StringIO(s.decode('utf-8')))
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
# Merge coorinates with df to get latitudes and longitudes for Postal Codes

df = pd.merge(coordinates, df, on='Postal Code')

# reorder column names and show the dataframe
df = df[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df.head(12)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
